In [ ]:
import os

dataset_path = r"C:\Users\Gauri Fargade\Desktop\jupyter projects\.ipynb_checkpoints"

if os.path.exists(dataset_path):
    print(f"Dataset folder found at: {dataset_path}")
    
else:
    print("Dataset folder NOT found! Please check the path.")


In [ ]:
import os

train_folder = os.path.join(dataset_folder, "TRAIN")
test_folder = os.path.join(dataset_folder, "TEST")
train_images = os.listdir(train_folder) if os.path.exists(train_folder) else []
test_images = os.listdir(test_folder) if os.path.exists(test_folder) else []

print(f" Number of images in TRAIN folder: {len(train_images)}")
print(f" Number of images in TEST folder: {len(test_images)}")


print(" Train Folder Images:", os.listdir(train_folder))
print(" Test Folder Images:", os.listdir(test_folder))


In [ ]:
train_o_images = os.listdir(train_o_path)
train_r_images = os.listdir(train_r_path)

test_o_images = os.listdir(test_o_path)
test_r_images = os.listdir(test_r_path)

print(f" Number of images in TRAIN/O: {len(train_o_images)}")
print(f" Number of images in TRAIN/R: {len(train_r_images)}")
print(f" Number of images in TEST/O: {len(test_o_images)}")
print(f" Number of images in TEST/R: {len(test_r_images)}")

In [ ]:
import numpy as np
import cv2
from tqdm import tqdm


IMG_SIZE = 128  


def load_images_from_folder(folder_path, label):
    images = []
    labels = []
    for filename in tqdm(os.listdir(folder_path), desc=f"Processing {label} images"):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)  # Read image
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize
            img = img / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(label)
    return images, labels


train_o_images, train_o_labels = load_images_from_folder(train_o_path, label=0)  # Organic = 0
train_r_images, train_r_labels = load_images_from_folder(train_r_path, label=1)  # Reusable = 1


test_o_images, test_o_labels = load_images_from_folder(test_o_path, label=0)
test_r_images, test_r_labels = load_images_from_folder(test_r_path, label=1)


X_train = np.array(train_o_images + train_r_images)
y_train = np.array(train_o_labels + train_r_labels)

X_test = np.array(test_o_images + test_r_images)
y_test = np.array(test_o_labels + test_r_labels)


np.save("train_data_preprocessed.npy", X_train)
np.save("train_labels_encoded.npy", y_train)
np.save("test_data_preprocessed.npy", X_test)
np.save("test_labels_encoded.npy", y_test)

print("✅ Image preprocessing complete!")



In [ ]:

!pip install tensorflow keras

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values (0-1)
    validation_split=0.2  # Splitting 20% of training data for validation
)


dataset_path = r"C:\Users\Gauri Fargade\Desktop\jupyter projects\.ipynb_checkpoints\DATASET\TRAIN"


train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=32,  
    class_mode="binary",
    subset="training"
)


val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=32,  
    class_mode="binary",
    subset="validation"
)

print(" Data is now loaded using generators! No MemoryError!")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Prevents overfitting
    Dense(1, activation='sigmoid')  # Binary classification (O vs R)
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values
    rotation_range=20,   # Random rotation
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Only rescale for validation


train_generator = train_datagen.flow_from_directory(
    'C:/Users/Gauri Fargade/Desktop/jupyter projects/.ipynb_checkpoints/DATASET/TRAIN',
    target_size=(128, 128),  # Resize images
    batch_size=32,
    class_mode='binary'  # Use 'categorical' if you have more than 2 classes
)

validation_generator = val_datagen.flow_from_directory(
    'C:/Users/Gauri Fargade/Desktop/jupyter projects/.ipynb_checkpoints/DATASET/TEST',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

print(" Data generators are ready!")

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # Adjust as needed
    validation_data=validation_generator
)`

# Save model
model.save("waste_classification_model.h5")

print("Training complete! Model saved.")


In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

# Load trained model
model = load_model("waste_classification_model.h5")

# Define path to the test dataset
test_dir = "C:/Users/Gauri Fargade/Desktop/jupyter projects/.ipynb_checkpoints/DATASET/TEST"

# Define the class labels
class_labels = {0: "Organic Waste", 1: "Recyclable Waste"}  # Adjust if needed

# Function to preprocess an image for prediction
def preprocess_image(image_path, target_size=(128, 128)):
    img = load_img(image_path, target_size=target_size)  # Load image
    img = img_to_array(img) / 255.0  # Convert to array and normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Get all images in the TEST directory
for category in ["O", "R"]:  # 'O' for Organic, 'R' for Recyclable
    category_path = os.path.join(test_dir, category)
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        
        # Preprocess the image
        img = preprocess_image(img_path)
        
        # Predict the class
        prediction = model.predict(img)
        predicted_class = 1 if prediction[0][0] > 0.5 else 0  # 0: Organic, 1: Recyclable
        
        # Display the result
        plt.figure()
        plt.imshow(load_img(img_path))  # Show image
        plt.title(f"Predicted: {class_labels[predicted_class]}\nActual: {class_labels[1] if category == 'R' else class_labels[0]}")
        plt.axis("off")
        plt.show()

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os

# Load the trained model
model = tf.keras.models.load_model("waste_classification_model.h5")

# Define class labels (assuming 0 = Organic, 1 = Reusable)
class_labels = {0: "Organic", 1: "Reusable"}

# Function to preprocess an image
def preprocess_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)  # Read the image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, target_size)  # Resize image to match model input
    img = img / 255.0  # Normalize pixel values (0-1)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Path to the folder containing test images
test_folder = "C:/Users/Gauri Fargade/Desktop/jupyter projects/.ipynb_checkpoints/DATASET/TEST"

# Loop through test images
for category in ["O", "R"]:  # Folders for Organic (O) and Reusable (R)
    category_path = os.path.join(test_folder, category)
    for image_name in os.listdir(category_path):
        image_path = os.path.join(category_path, image_name)
        image_data = preprocess_image(image_path)

        # Predict class (0 or 1)
        prediction = model.predict(image_data)
        predicted_class = 1 if prediction[0][0] > 0.5 else 0  # Threshold at 0.5

        # Print result
        print(f"Image: {image_name} → Predicted: {predicted_class} ({class_labels[predicted_class]})")


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

# Load the trained model
model = tf.keras.models.load_model("waste_classification_model.h5")

# Define class labels (assuming 0 = Organic, 1 = Reusable)
class_labels = {0: "Organic", 1: "Reusable"}

# Function to preprocess an image
def preprocess_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)  # Read the image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, target_size)  # Resize image to match model input
    img = img / 255.0  # Normalize pixel values (0-1)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Path to the folder containing test images
test_folder = "C:/Users/Gauri Fargade/Desktop/jupyter projects/.ipynb_checkpoints/DATASET/TEST"

# Loop through test images
for category in ["O", "R"]:  # Folders for Organic (O) and Reusable (R)
    category_path = os.path.join(test_folder, category)
    image_names = sorted(os.listdir(category_path))  # Sort images in order
    
    # Select first 3 and last 2 images
    selected_images = image_names[:3] + image_names[-2:]

    print(f"\nPredictions for category: {category}")

    fig, axes = plt.subplots(1, len(selected_images), figsize=(15, 5))  # Create subplots

    for i, image_name in enumerate(selected_images):
        image_path = os.path.join(category_path, image_name)
        image_data = preprocess_image(image_path)

        # Predict the class
        prediction = model.predict(image_data)
        predicted_class = int(round(prediction[0][0]))  # Convert to 0 or 1

        # Read the image for displaying
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Show the image with prediction
        axes[i].imshow(img)
        axes[i].set_title(f"{image_name}\nPredicted: {class_labels[predicted_class]}")
        axes[i].axis("off")

    plt.show()  # Display the images


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

# Load the trained model
model = tf.keras.models.load_model("waste_classification_model.h5")

# Define class labels
class_labels = {0: "Organic", 1: "Reusable"}

# Function to preprocess an image
def preprocess_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)  # Read the image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, target_size)  # Resize image to match model input
    img = img / 255.0  # Normalize pixel values (0-1)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Path to the folder containing test images
test_folder = r"C:\Users\Gauri Fargade\Pictures\test"

# Get the list of image files in the folder
image_files = sorted(os.listdir(test_folder))  # Sort to ensure correct order

# Loop through all images in the folder
for image_name in image_files:
    image_path = os.path.join(test_folder, image_name)
    
    # Preprocess and predict
    image_data = preprocess_image(image_path)
    prediction = model.predict(image_data)
    
    # Convert prediction to 0 or 1
    predicted_class = int(prediction[0][0] > 0.5)  # 0 = Organic, 1 = Reusable
    
    # Display image with prediction
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Predicted: {class_labels[predicted_class]}")
    plt.show()
